### Abalone Regression in Pytorch

In [ ]:
# import libraries
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

# check GPUs
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# check Torch version
print(f"Torch version: {torch.__version__}")

# check modified date
print(f"Modified date: {datetime.now()}")


#### 1. Dataset 

In [ ]:
abalone_data = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

abalone_data.head()

In [ ]:
# get data and convert to numpy
X = abalone_data.drop("Age", axis=1).to_numpy()
X

In [ ]:
# get data and convert to numpy
y = abalone_data["Age"].to_numpy()
y

In [ ]:
# split data
split_amount = int(0.8* len(X))

# 80% of dataset 
X_train, y_train = torch.Tensor(X[:split_amount]), torch.Tensor(y[:split_amount])

# 20% of dataset
X_test, y_test = torch.Tensor(X[split_amount:]), torch.Tensor(y[split_amount:])

# check length
len(X_train), len(y_train), len(X_test), len(y_test)


In [ ]:
# check shapes of traning set
X_train.shape, y_train.shape

In [ ]:
# check shapes of test set
X_test.shape, y_test.shape

#### 2. Modeling

In [ ]:
class AbaloneRegressionNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = nn.Linear(in_features=7, out_features=1)
    
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return self.linear(X)

In [ ]:
# initialize a model
abalone_model = AbaloneRegressionNeuralNetwork()

# copy model to device
abalone_model.to(device)

# copy data to device
train_data = X_train.to(device)
labels_data = y_train.to(device)
test_data = X_test.to(device)
labels_data = y_test.to(device)

# print out weights and bias
abalone_model.state_dict()

# losses
loss_fn = nn.L1Loss()

# optimizer
optimizer = torch.optim.SGD(params = abalone_model.parameters(), lr=0.01)

In [ ]:
# start training on gpu

# tensorboard
writer = SummaryWriter()

# set manual seed
torch.cuda.manual_seed(42)

# epochs
epochs = 200

for epoch in range(epochs):

    ## TRAINING 

    # train mode
    abalone_model.train()

    # forward 
    y_preds = abalone_model(train_data)

    # calculate loss
    loss = loss_fn(y_preds, labels_data)

    # writing tensorboard 
    writer.add_scalar("Loss/train", loss, epoch)

    # reset zero_grad
    optimizer.zero_grad()

    # backward
    loss.backward()
    
    # step gradient_descent
    optimizer.step()

    # eval mode
    abalone_model.eval()

    ## EVALUATION
    with torch.inference_mode():

        # forward
        y_test_preds = abalone_model(test_data)

        # calculate loss
        test_loss = loss_fn(y_test_preds, labels_data)
        
        # print out
        if epoch % 10 == 0:
            print(f"Epoch: {epoch} | MAE loss: {loss} | Test loss: {test_loss}")

# to make sure all pending evens have been writien to dish 
writer.flush()
writer.close()
    

#### 3. Evaluation

In [ ]:
test_data[0], y_test[0]

In [ ]:
# eval mode
abalone_model.eval()

with torch.inference_mode():
    y_test_preds = abalone_model(test_data[0])

y_test_preds

In [ ]:
# release models and GPU
del abalone_model
torch.cuda.empty_cache()

#### 4. Visualizing TensorBoard

> https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html

In [ ]:
# !tensorboard --logdir=runs